### Setting Up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786_Project/

Mounted at /content/drive
/content/drive/MyDrive/ECE1786_Project


In [ ]:
!pip install wandb datasets transformers sentencepiece



In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Importing Data

In [ ]:
import pandas as pd
Training = pd.read_csv('Training.csv')
Validation = pd.read_csv('Validation.csv')
Test = pd.read_csv('Test.csv')

In [ ]:
Training.head()

,text,code
0,3 Write a program to find the element that app...,def bubbleSort(array):\n swapped = False\n ...
1,5 write a python program to find the third max...,def bubbleSort(array):\n swapped = False\n ...
2,6 Write a program to find the maximum product ...,def bubbleSort(array):\n swapped = False\n ...
3,7 write a python program to find the largest p...,def bubbleSort(array):\n swapped = False\n ...
4,8 write a Python program to print an array of ...,def bubbleSort(array):\n swapped = False\n ...


In [ ]:
Validation.head()

,text,code
0,1 write a python program to sort an array in a...,def bubbleSort(array):\n swapped = False\n ...
1,12 write a python program to sort an array in ...,def selectionSort(array):\n for i in range(...
2,23 write a python program to find the element ...,def insertionSort(array):\n for i in range(...
3,34 write a python program to check if any valu...,import math\ndef shellSort(array):\n n = le...
4,45 write a python program to find the third di...,"def heapify(array, n, i):\n smallest = i\n ..."


In [ ]:
Test.head()

,text,code
0,2 Write python code to sort an array in descen...,def bubbleSort(array):\n swapped = False\n ...
1,4 write a python program to check if any value...,def bubbleSort(array):\n swapped = False\n ...
2,13 Write python code to determine the element ...,def selectionSort(array):\n for i in range(...
3,15 Use the Selection Sort algorithm to find th...,def selectionSort(array):\n for i in range(...
4,24 Write python code to check if any value app...,def insertionSort(array):\n for i in range(...


In [ ]:
from datasets import Dataset
training_set = Dataset.from_pandas(Training)
validation_set = Dataset.from_pandas(Validation)
test_set = Dataset.from_pandas(Test)

In [ ]:
training_set

Dataset({
    features: ['text', 'code'],
    num_rows: 202
})

In [ ]:
validation_set

Dataset({
    features: ['text', 'code'],
    num_rows: 28
})

In [ ]:
test_set

Dataset({
    features: ['text', 'code'],
    num_rows: 86
})

### Model Definition

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

model = AutoModelWithLMHead.from_pretrained("Salesforce/codet5-base")

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/703k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

### Pre-Processing

In [ ]:
def prepare_data(examples):
    text = examples['text']
    code = examples['code']
    
    model_inputs = tokenizer(text, padding="longest", truncation = True)
    labels = tokenizer(code, padding="longest", truncation = True).input_ids

    labels_with_ignore_index = []
    for labels_example in labels:
      labels_example = [label if label != 0 else -100 for label in labels_example]
      labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs

In [ ]:
tokenized_training_set = training_set.map(prepare_data, batched=True, remove_columns=training_set.column_names)
tokenized_validation_set = validation_set.map(prepare_data, batched=True, remove_columns=validation_set.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
labels = tokenized_training_set["labels"][0]
print(tokenizer.decode([label for label in labels if label != -100]))

<s>def bubbleSort(array):
    swapped = False
    for i in range(len(array)-1,0,-1):
        for j in range(i):
            if array[j]>array[j+1]:
                array[j], array[j+1] = array[j+1], array[j]
                swapped= True
        if swapped:
            swapped=False
        else:
            break
    return array
arr = [2,2,1,1,1,2,2]
arr = bubbleSort(arr)
print(arr[len(arr)//2])








</s>


### Training & Saving

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "/content/drive/MyDrive/ECE1786_Project/results",
    logging_steps = 51,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="wandb"
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_training_set,
    eval_dataset=tokenized_validation_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
wandb.init(project="codet5_base_finetuning", entity="dhairya18498")

wandb: Currently logged in as: dhairya18498. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 202
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 510
  Number of trainable parameters = 222882048
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,2.851300,1.178101
2,1.131900,0.610449
3,0.697800,0.341731
4,0.478800,0.234345
5,0.347000,0.165846
6,0.264200,0.113512
7,0.211900,0.098326
8,0.173000,0.084236
9,0.149600,0.082119
10,0.137900,0.080288


***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
Saving model checkpoint to /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500
Configuration saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/tokeni

TrainOutput(global_step=510, training_loss=0.644329776950911, metrics={'train_runtime': 387.1297, 'train_samples_per_second': 5.218, 'train_steps_per_second': 1.317, 'total_flos': 105711283814400.0, 'train_loss': 0.644329776950911, 'epoch': 10.0})

In [ ]:
from pathlib import Path
model_path = Path('/content/drive/MyDrive/ECE1786_Project/code_t5_finetuned_sorting.pt')

In [ ]:
torch.save(model, model_path)

In [ ]:
trainer.save_model("codet5_base_finetuned_on_sorting_algorithms")

Saving model checkpoint to codet5_base_finetuned_on_sorting_algorithms
Configuration saved in codet5_base_finetuned_on_sorting_algorithms/config.json
Model weights saved in codet5_base_finetuned_on_sorting_algorithms/pytorch_model.bin
tokenizer config file saved in codet5_base_finetuned_on_sorting_algorithms/tokenizer_config.json
Special tokens file saved in codet5_base_finetuned_on_sorting_algorithms/special_tokens_map.json


In [ ]:
model = AutoModelWithLMHead.from_pretrained("codet5_base_finetuned_on_sorting_algorithms")

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file codet5_base_finetuned_on_sorting_algorithms/config.json
Model config T5Config {
  "_name_or_path": "codet5_base_finetuned_on_sorting_algorithms",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
 

### Evaluation

In [ ]:
import random

In [ ]:
random.seed(1)
random_idx = random.choice(list(Test.index))
sample_input = Test["text"][random_idx]
true_label = Test["code"][random_idx]
print('#sample_input:', sample_input, '\n')
print('#true_label (True Code):')
print(true_label, '\n')

#sample_input: 84 write a python program that uses Radix Sort algorithm to determine if any value appears at least twice in the array. 

#true_label (True Code):
import itertools
def radixSort(array):
    min_num = min(array)
    if min_num < 0:
        lb = abs(min_num)
        array = [item + lb for item in array]
    n_digits = len(str(max(array)))
    for dgt in range(n_digits):
        buckets = [[] for i in range(10)]
        for num in array:
            idx = (num // (10**dgt)) % 10
            buckets[idx].append(num)
        array = list(itertools.chain(*buckets))
    if min_num < 0:
        lb = abs(min_num)
        array = [item - lb for item in array]
    return array
arr = [1,2,3,1]
arr = radixSort(arr)
for i in range(len(arr)-1):
    if arr[i] == arr[i+1]:
        print("A value appears at least twice in the array")
        break
    else:
        if (i+1 == len(arr)-1):
            print("No value appear twice in the array")




 



In [ ]:
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=300)
print('#generated output:')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#generated output:
import itertools
def radixSort(array):
    min_num = min(array)
    if min_num < 0:
        lb = abs(min_num)
        array = [item + lb for item in array]
    n_digits = len(str(max(array)))
    for dgt in range(n_digits):
        buckets = [[] for i in range(10)]
        for num in array:
            idx = (num // (10**dgt)) % 10
            buckets[idx].append(num)
        array = list(itertools.chain(*buckets))
    if min_num < 0:
        lb = abs(min_num)
        array = [item - lb for item in array]
    return array
arr = [1,2,3,1]
arr = radixSort(arr)
for i in range(len(arr)-1):
    if arr[i] == arr[i+1]:
        print("A value appears at least twice in the array")
        break
    else:
        if (i+1 == len(arr)-1):
            print("No value appear twice in the array")









In [ ]:
#generated output:
import itertools
def radixSort(array):
    min_num = min(array)
    if min_num < 0:
        lb = abs(min_num)
        array = [item + lb for item in array]
    n_digits = len(str(max(array)))
    for dgt in range(n_digits):
        buckets = [[] for i in range(10)]
        for num in array:
            idx = (num // (10**dgt)) % 10
            buckets[idx].append(num)
        array = list(itertools.chain(*buckets))
    if min_num < 0:
        lb = abs(min_num)
        array = [item - lb for item in array]
    return array
arr = [1,2,3,1]
arr = radixSort(arr)
for i in range(len(arr)-1):
    if arr[i] == arr[i+1]:
        print("A value appears at least twice in the array")
        break
    else:
        if (i+1 == len(arr)-1):
            print("No value appear twice in the array")


A value appears at least twice in the array


In [ ]:
sample_input = "With Gnome Sort, sort an array in non-descending order and print the second last element"
print('#sample input: ', sample_input, '\n')
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=300)
print('#generated output:')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#sample input:  With Gnome Sort, sort an array in non-descending order and print the second last element 

#generated output:
def gnomeSort(arr):
    n = len(arr)
    index = 0
    while index < n:
        if index == 0:
            index = index + 1
        if arr[index] >= arr[index - 1]:
            index = index + 1
        else:
            arr[index], arr[index-1] = arr[index-1], arr[index]
            index = index - 1
 
    return arr


arr = [2,2,3,1]
arr = gnomeSort(arr)
print(arr[len(arr)-1])









In [ ]:
#sample input:  With Gnome Sort, sort an array in non-descending order and print the second last element 

#generated output:
def gnomeSort(arr):
    n = len(arr)
    index = 0
    while index < n:
        if index == 0:
            index = index + 1
        if arr[index] >= arr[index - 1]:
            index = index + 1
        else:
            arr[index], arr[index-1] = arr[index-1], arr[index]
            index = index - 1
 
    return arr


arr = [2,2,3,1]
arr = gnomeSort(arr)
print(arr)

[1, 2, 2, 3]
